In [ ]:
# ==============================================================================
# NOTEBOOK DE TREINAMENTO YOLOv8 COM DOWNLOAD DO ROBOFLOW
# ==============================================================================

# 1. INSTALAR AS BIBLIOTECAS NECESSÁRIAS
# ------------------------------------------------------------------------------
print("Instalando Ultralytics e Roboflow...")
!pip install -q ultralytics roboflow
print("Instalação concluída.")

In [ ]:
# 2. IMPORTAÇÕES E AUTENTICAÇÃO NO ROBOFLOW
# ------------------------------------------------------------------------------
import os
from ultralytics import YOLO
from roboflow import Roboflow
import shutil

print("Autenticando no Roboflow...")
rf = Roboflow(api_key="kpItsaAjjSHi7wIt4BHe")
print("Autenticação bem-sucedida.")

In [ ]:
# 3. DOWNLOAD DO DATASET DO ROBOFLOW
# ------------------------------------------------------------------------------
print("Baixando o dataset do Roboflow...")
try:
    project = rf.workspace("florian-demolder").project("recognition-of-direction-arrows-for-autonomus-car")
    version = project.version(4)
    dataset = version.download("yolov8")

    # Guarda o caminho para o arquivo de configuração, que é essencial para o treino
    DATASET_CONFIG_PATH = os.path.join(dataset.location, "data.yaml")
    print(f"Dataset baixado com sucesso em: {dataset.location}")
    print(f"Arquivo de configuração encontrado em: {DATASET_CONFIG_PATH}")
except Exception as e:
    print(f"ERRO: Falha ao baixar o dataset. Verifique sua chave de API e o nome do projeto. Detalhes: {e}")

In [ ]:
# 4. TREINAMENTO DO MODELO YOLOv8
# ------------------------------------------------------------------------------
# Carrega o modelo 'yolov8n.pt'. 'n' = nano, o menor e mais rápido, ideal para o RPi.
# Ele usa "Transfer Learning" por padrão, pois já foi pré-treinado.
model = YOLO('yolov8n.pt')

print("\nIniciando o treinamento do YOLOv8...")
# Inicia o treinamento usando o arquivo de configuração do dataset baixado
results = model.train(
    data=DATASET_CONFIG_PATH,
    epochs=50,
    imgsz=640,       # Tamanho da imagem para o treinamento
    patience=5,      # Para o treino se não houver melhora por 5 épocas
    name='yolov8n_directions_training' # Nome para a pasta de resultados
)
print("Treinamento concluído!")

In [ ]:
# 5. EXPORTAÇÃO E DOWNLOAD DO MELHOR MODELO
# ------------------------------------------------------------------------------
print("\nExportando e preparando o modelo para download...")
# O melhor modelo é salvo como 'best.pt' na pasta de resultados do treino
best_model_path = results.save_dir / 'weights' / 'best.pt'

# Verifica se o arquivo existe antes de tentar fazer o download
if os.path.exists(best_model_path):
    print(f"Melhor modelo encontrado em: {best_model_path}")

    # Faz o download do arquivo .pt
    from google.colab import files
    print("Iniciando o download do arquivo 'best.pt'... Aguarde a janela do navegador.")
    files.download(best_model_path)
else:
    print(f"ERRO: O modelo treinado ('best.pt') não foi encontrado no caminho esperado.")